# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [359]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [360]:
orders = pd.read_csv("Orders.csv")
orders.drop(['Unnamed: 0'], axis=1, inplace=True)
orders.columns= orders.columns.str.lower().str.replace(' ', '_')
orders.head()

,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [371]:
# pd.set_option('display.max_rows',None)
customer_table = orders.groupby(['customerid', 'country']).agg({'amount_spent':sum}).reset_index()
customer_table.shape

(4347, 3)

In [362]:
customer_table['customerid'].nunique()
# so there are 8 customers which are active in more than one country

4339

In [363]:
# list all those customers
customer_table[customer_table.duplicated(subset=['customerid'],keep=False)]


,customerid,country,amount_spent
20,12370,Austria,277.20
21,12370,Cyprus,3268.49
39,12394,Belgium,381.08
40,12394,Denmark,891.40
58,12417,Belgium,3212.80
59,12417,Spain,436.30
63,12422,Australia,386.20
64,12422,Switzerland,417.36
71,12429,Austria,437.98
72,12429,Denmark,3312.42


In [364]:
# add there amount_spent to their major country
j=0
droprows = []
for i in customer_table[customer_table.duplicated(subset=['customerid'],keep=False)].index:    
    if j % 2 == 0:
        if customer_table.iloc[i,2] >= customer_table.iloc[i+1,2]:
            customer_table.iloc[i,2] = customer_table.iloc[i,2] + customer_table.iloc[i+1,2] 
            droprows.append(i+1)            
        else:
            customer_table.iloc[i+1,2] = customer_table.iloc[i,2] + customer_table.iloc[i+1,2] 
            droprows.append(i)                        
    j = j+1        

# drop duplictaed customer entries
for i in droprows[::-1]:
    customer_table.drop(i, inplace=True)

In [365]:
# create 20 equally filled groups 
labels = []
for i in range(20):    
    labels.append(i*5)    
customer_table['percentiles'] = pd.qcut(customer_table['amount_spent'], 20, labels = labels)
customer_table = customer_table[customer_table.percentiles >= 75]

# create two customer groups
customer_table['cgroup'] = np.where(customer_table['percentiles'] < 95, "Preferred Customers", "VIP Customer")
customer_table


C:\Users\TgMiles\AppData\Local\Temp/ipykernel_19472/2345015204.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_table['cgroup'] = np.where(customer_table['percentiles'] < 95, "Preferred Customers", "VIP Customer")


,customerid,country,amount_spent,percentiles,cgroup
0,12346,United Kingdom,77183.60,95,VIP Customer
1,12347,Iceland,4310.00,90,Preferred Customers
2,12348,Finland,1797.24,75,Preferred Customers
3,12349,Italy,1757.55,75,Preferred Customers
5,12352,Norway,2506.04,80,Preferred Customers
...,...,...,...,...,...
4327,18259,United Kingdom,2338.60,80,Preferred Customers
4328,18260,United Kingdom,2643.20,85,Preferred Customers
4336,18272,United Kingdom,3078.58,85,Preferred Customers
4345,18283,United Kingdom,2094.88,80,Preferred Customers


In [366]:
customer_table['cgroup'].value_counts()

Preferred Customers    868
VIP Customer           217
Name: cgroup, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [367]:
customer_table['country'][customer_table['cgroup']=='VIP Customer'].value_counts()    

United Kingdom     177
Germany             10
France               9
Switzerland          3
Australia            2
Spain                2
Japan                2
Portugal             2
EIRE                 2
Finland              1
Norway               1
Denmark              1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [368]:
customer_table['country'].value_counts()    

United Kingdom     932
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Portugal             7
Norway               7
Italy                5
Finland              5
Channel Islands      4
Australia            4
Japan                4
Cyprus               3
Denmark              3
EIRE                 3
Israel               2
Sweden               2
Singapore            1
Lebanon              1
Poland               1
Iceland              1
Greece               1
Netherlands          1
Austria              1
Canada               1
Malta                1
Name: country, dtype: int64